Importing relevant packages and modules

In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [2]:
import sys
from pathlib import Path

# Plotting
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('bioAI.mplstyle')
import copy

'''
CUSTOM PACKAGES
'''
# avoid adding multiple relave paths to sys.path
sys.path.append("../src") if "../src" not in sys.path else None

from plotting_functions import *
from synthetic_grid_cells import GridModule, rotation_matrix
from methods import multiimshow, PCA_UMAP, filenames
from Experiment import Experiment
from stats import *

In [3]:
#base_path = Path("/storA/GridCells/")
base_path = "/mnt/WD12TB-HDD"
experiment = Experiment(name="gg-3ME", base_path=base_path)
experiment.setup()
paths = experiment.paths
boxsize = experiment.environments[0].boxsize

Experiment <gg-3ME> already EXISTS. Loading experiment settings!
Loading experiment details
This experiment has ALREADY been setup - SKIPPING.


In [4]:
module_indices = np.load(f"{experiment.paths['experiment']}/module_indices_all_clusters.npz")
clusters = [module_indices[f] for f in module_indices.files]

torus_cells = list(clusters[2]) + list(clusters[12]) + list(clusters[15])
torus_cells.sort()
clustered_grid_cells = clusters[1]
border_cells = list(clusters[3]) + list(clusters[8])
border_cells.sort()
place_cells = clusters[10]
# include the rest of the cells in the unsorted cells (18 clusters total)
unsorted_cells = list(clusters[0]) + list(clusters[4]) + list(clusters[5]) + list(clusters[6]) + list(clusters[7]) + \
    list(clusters[9]) + list(clusters[11]) + list(clusters[13]) + list(clusters[14]) + list(clusters[16]) + list(clusters[17])
unsorted_cells.sort()
print(f"{len(torus_cells)=}, {len(clustered_grid_cells)=}, {len(border_cells)=}, {len(place_cells)=}, {len(unsorted_cells)=}")
clusters = [torus_cells, clustered_grid_cells, border_cells, place_cells, unsorted_cells]
cluster_labels = ["torus", "clustered grid", "border", "place", "unsorted"]

len(torus_cells)=604, len(clustered_grid_cells)=85, len(border_cells)=167, len(place_cells)=103, len(unsorted_cells)=2172


In [10]:
ratemaps = []
for env_i in range(3):
    with open(paths['ratemaps'] / (f'env_{env_i}/' + filenames(paths['ratemaps'] / f'env_{env_i}')[-1]), 'rb') as f:
        tmp_ratemaps = pickle.load(f)
        # fix nans
        tmp_ratemaps = np.array([fill_nan(ratemap, method='nearest') for ratemap in tmp_ratemaps])
        ratemaps.append(tmp_ratemaps)
ratemaps = np.stack(ratemaps)
torus_ratemaps = ratemaps[:, torus_cells]

(3, 604, 64, 64)

In [11]:
# split torus cluster into three band cell clusters
band_classes = cluster_ratemaps(torus_ratemaps[0]) # assume ratemaps are torus ratemaps here
band_cells = [np.array(torus_cells)[band_classes == i] for i in range(3)]

100%|██████████| 604/604 [00:28<00:00, 21.13it/s]


In [14]:
from methods import multiimshow2
for b, band_cells_i in enumerate(band_cells):
    for env_i in range(3):
        fig, axs = multiimshow2(ratemaps[env_i][band_cells_i], normalize=False, figsize=(4,4), rect=(0,0,1,1), axes_pad=0.02)
        fig.savefig(experiment.paths['experiment'] / f"plots/B{b}_ratemaps_{env_i}")

IndexError: index 615 is out of bounds for axis 0 with size 604

In [13]:
band_cells_i

array([   2,   16,   27,   67,   73,  119,  181,  185,  193,  206,  257,
        259,  260,  300,  302,  314,  378,  390,  423,  426,  471,  475,
        495,  541,  555,  615,  668,  683,  707,  725,  758,  761,  774,
        779,  803,  808,  823,  947,  973, 1118, 1146, 1213, 1225, 1237,
       1292, 1311, 1371, 1372, 1392, 1423, 1430, 1453, 1468, 1479, 1482,
       1507, 1547, 1591, 1595, 1598, 1613, 1652, 1656, 1658, 1691, 1741,
       1767, 1784, 1836, 1869, 1904, 1914, 1941, 1951, 1961, 1976, 2042,
       2110, 2137, 2195, 2198, 2216, 2227, 2228, 2235, 2255, 2256, 2267,
       2269, 2340, 2350, 2369, 2392, 2410, 2422, 2448, 2454, 2485, 2491,
       2498, 2632, 2636, 2664, 2668, 2672, 2745, 2746, 2809, 2819, 2835,
       2873, 2876, 2890, 2904, 2924, 2933, 2953, 2966, 3030, 3041, 3065,
       3183, 3185, 3193, 3221, 3355, 3365, 3375, 3394, 3413, 3417, 3502,
       3513, 3526, 3530, 3531, 3566, 3595, 3653, 3721, 3758, 3759, 3791,
       3812, 3824, 3875, 3908, 3931, 3935, 3949, 39